In [2]:
import torch
import random




In [43]:
BATCHSIZE = 100
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  r = torch.clamp(r,0,1)
  return torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)

for i in range(BATCHSIZE):
  s = getSample()
  X = torch.cat((X, s), dim=0)

for i in X:
  s = dif(i, TARGET)
  Y = torch.cat((Y, s), dim=0)
  


list(zip(X, Y))


[(tensor([0.1857, 0.4319], dtype=torch.float64),
  tensor([0.3862], dtype=torch.float64)),
 (tensor([0.0928, 0.0522], dtype=torch.float64),
  tensor([0.3950], dtype=torch.float64)),
 (tensor([0.2200, 0.4238], dtype=torch.float64),
  tensor([0.3438], dtype=torch.float64)),
 (tensor([0.4286, 0.9041], dtype=torch.float64),
  tensor([0.7927], dtype=torch.float64)),
 (tensor([0.7218, 0.0085], dtype=torch.float64),
  tensor([0.5733], dtype=torch.float64)),
 (tensor([0.5329, 0.5311], dtype=torch.float64),
  tensor([0.5240], dtype=torch.float64)),
 (tensor([0.6375, 0.5750], dtype=torch.float64),
  tensor([0.6725], dtype=torch.float64)),
 (tensor([0.1635, 0.2229], dtype=torch.float64),
  tensor([0.1994], dtype=torch.float64)),
 (tensor([0.0247, 0.6596], dtype=torch.float64),
  tensor([0.7749], dtype=torch.float64)),
 (tensor([0.0554, 0.0596], dtype=torch.float64),
  tensor([0.4250], dtype=torch.float64)),
 (tensor([0.3629, 0.5407], dtype=torch.float64),
  tensor([0.3637], dtype=torch.float64)),

In [44]:
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf


# thus, we rate the variables accordingly: A value closer to 0.34, 0.2 is rated with a lower number


train_Y = standardize(Y)


gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)


UCB = UpperConfidenceBound(gp, beta=0.1)


bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

print(candidate)



tensor([[0.0000, 0.9986]])


In [ ]:
def optimize_qehvi(model, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_obj)
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200, "nonnegative": True},
        sequential=True,
    )
    # observe new values
    new_x =  unnormalize(candidates.detach(), bounds=problem_bounds)
    return new_x